In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.io as scio
import matlab

np.set_printoptions(threshold=10000)
plt.rcParams['text.usetex'] = True
# plt.rcParams['font.sans-serif'] = 'Times New Roman'
# plt.rcParams['mathtext.fontset'] = 'cm'

In [2]:
# 导入数据(1)：外生变量
M = 31
TL1 = 10
TL2 = 5
TL = TL1 + TL2

## 2002年的土地比例：住宅/(商服+工业)
LandShare = pd.read_excel('./data/LandShare.xlsx')
LandShare = np.array(LandShare.iloc[0:31, 4]).reshape([31, 1])

## 用2002年建设用地总面积和上面的比例求取2002年的工业用地存量和住宅用地存量
L_02 = pd.read_excel('./data/LandArea.xlsx')
L_02 = np.array(L_02.iloc[0:31, 2]).reshape([31, 1])
Lp_02 = np.array(L_02).reshape([31, 1]) / (1 + LandShare) # 工业用地
Lh_02 = L_02 - Lp_02 # 住宅用地
Lp_02 = np.delete(Lp_02, 25, axis=0) # 删除西藏
Lh_02 = np.delete(Lh_02, 25, axis=0) # 删除西藏
Lp_02_world = (pd.read_excel('./data/国家面积.xlsx').iloc[43, 1] / pd.read_excel('./data/国家面积.xlsx').iloc[43, 2] - 1) * Lp_02.sum() # 利用比例估计外国的工业用地面积
Lh_02_world = (pd.read_excel('./data/国家面积.xlsx').iloc[43, 1] / pd.read_excel('./data/国家面积.xlsx').iloc[43, 2] - 1) * Lh_02.sum() # 利用比例估计外国的住宅用地面积
Lp_02 = np.insert(Lp_02, 30, Lp_02_world, axis=0).reshape([31, 1]) # 加入外国
Lh_02 = np.insert(Lh_02, 30, Lh_02_world, axis=0).reshape([31, 1]) # 加入外国

## 03至17年的工业用地存量和住宅用地存量
L_0317 = pd.read_excel('./data/LandArea.xlsx')
L_0317 = np.array(L_0317.iloc[:, 3:]) 
Lp_0317 = L_0317 / ((LandShare + 1) @ np.ones([1, 15])) # 用02年的比例直接计算后面年份的用地存量
Lh_0317 = L_0317 - Lp_0317
Lp_0317 = np.delete(Lp_0317, 25, axis=0) # 剔除西藏
Lh_0317 = np.delete(Lh_0317, 25, axis=0) # 剔除西藏
Lp_0317 = np.insert(Lp_0317, 30, Lp_02[-1]@np.ones([1, 15]), axis=0) # 加入外国，设定其保持02年值不变
Lh_0317 = np.insert(Lh_0317, 30, Lh_02[-1]@np.ones([1, 15]), axis=0) # 加入外国，设定其保持02年值不变

## 税收
Tau = 0.08 # 收入税
T = pd.read_excel('./data/TransFis.xlsx') # 企业税收
T = np.array(T.iloc[:, 22])
T = np.insert(T, 31, 0.1, axis=0) # 加入外国，设定税率是0.1
T = np.delete(T, 25, axis=0).reshape([31, 1]) # 删除西藏

## 转移支付
S_02 = pd.read_excel('./data/TransFis.xlsx') # 02年的转移支付比例
S_02 = np.array(S_02.iloc[:, 4]).reshape([31, 1])
S_02 = np.delete(S_02, 25, axis=0) # 删除西藏
S_02 = np.insert(S_02, 30, 0.0001, axis=0).reshape([31, 1]) # 加入外国，设定转移支付为0.0001非常小

S_0317 = pd.read_excel('./data/TransFis.xlsx') # 03年至17年的转移支付比例
S_0317 = np.array(S_0317.iloc[:, 5:20])
S_0317 = np.delete(S_0317, 25, axis=0) # 删除西藏
S_0317 = np.insert(S_0317, 30, 0.0001*np.ones([1, 15]), axis=0) # 加入外国

In [3]:
# 导入数据(2)：内生变量
## 2002年的移民份额
Mkj_02 = pd.read_excel('./data/2000省际人口流动_整理.xlsx')
Mkj_02 = np.array(Mkj_02.iloc[36:, 1:-2]).T
Mkj_02 = np.delete(Mkj_02, 25, axis=0)
Mkj_02 = np.delete(Mkj_02, 25, axis=1) 
Mkj_02 = Mkj_02 / Mkj_02.sum(axis=1).reshape([30, 1]) # 去掉西藏以后行求和不再是1了，因此需要按比例放大
Mkj_02 = np.insert(Mkj_02, 30, np.zeros([1, 30]), axis=1)
Mkj_02 = np.insert(Mkj_02, 30, np.append(np.zeros([1, 30]), 1), axis=0) # 加入外国，外国移民设定为0
Mkj_02 = Mkj_02.astype(float) # 将ndarray的type由object转换为float

## 2012年的移民份额
Mkj_12 = pd.read_excel('./data/2010省际人口流动_整理.xlsx')
Mkj_12 = np.array(Mkj_12.iloc[37:, 5:]).T
Mkj_12 = np.delete(Mkj_12, 25, axis=0)
Mkj_12 = np.delete(Mkj_12, 25, axis=1) 
Mkj_12 = Mkj_12 / Mkj_12.sum(axis=1).reshape([30, 1]) # 去掉西藏以后行求和不再是1了，因此需要按比例放大
Mkj_12 = np.insert(Mkj_12, 30, np.zeros([1, 30]), axis=1)
Mkj_12 = np.insert(Mkj_12, 30, np.append(np.zeros([1, 30]), 1), axis=0) # 加入外国，外国移民设定为0
Mkj_12 = Mkj_12.astype(float) # 将ndarray的type由object转换为float

## 构建2003-2012年的移民份额(用插值补充中间10年的移民份额)
Mkj_0312 = np.ndarray([31, 31, 10])
for t in range(0, 10):
    Mkj_0312[:, :, t] = Mkj_02 + (Mkj_12 - Mkj_02) / 10 * (t + 1)

## 各地居住居民数量
N_02 = pd.read_excel('./data/2000省际人口流动_整理.xlsx')
N_02 = np.array(N_02.iloc[2:33, -1]).reshape([31, 1])
N_02 = np.delete(N_02, 25, axis=0) # 删除西藏
N_02_world = pd.read_excel('./data/EmpGDPChnWorld.xlsx').iloc[17, 3] / pd.read_excel('./data/EmpGDPChnWorld.xlsx').iloc[0, 3] * N_02.sum() # 用中国与外国人口的比例来估计外国的居民数量
N_02 = np.insert(N_02, 30, N_02_world, axis=0) 
N0_02 = (N_02.T @ np.linalg.inv(Mkj_02)).T # 本地户籍人口

## 价格指数
P_02 = np.ones([M, 1]) # 设定初期所有地区价格指数均为1
P_0317 = pd.read_excel('./data/CPI.xlsx')
P_0317 = np.array(P_0317.iloc[2:17, 32:-1])
P_0317 = np.flip(P_0317, axis=0).T # 上下翻转，使得2002年在第一行，然后转置
P_0317 = np.delete(P_0317, 25, axis=0) # 删除西藏
P_0317 = np.insert(P_0317, 30, P_0317.mean(axis=0), axis=0) # 加入外国，外国物价水平设定为各地均值

## 税前实际收入
W_02 = pd.read_excel('./data/城镇人均可支配收入.xlsx') # 02年的人均可支配收入
W_02 = np.array(W_02.iloc[49, 1:]).reshape([31, 1]) / (1 - Tau) # 税前收入
W_02 = np.delete(W_02, 25, axis=0) # 删除西藏
W_02_world = pd.read_excel('./data/EmpGDPChnWorld.xlsx').iloc[17, 2] / pd.read_excel('./data/EmpGDPChnWorld.xlsx').iloc[0, 2] * (W_02.T @ N_02[:-1]) / N_02[-1] # 用中国与外国的GDP比例来估计外国的工资总额，然后除以前面估计的外国居民总数
W_02 = np.insert(W_02, 30, W_02_world, axis=0)
WoP_02 = W_02 / P_02 # 除以价格指数得到税前实际收入

W_0317 = pd.read_excel('./data/城镇人均可支配收入.xlsx') # 03年至17年的人均可支配收入
W_0317 = np.array(W_0317.iloc[50:65, 1:]).T / (1 - Tau) # 税前收入
W_0317 = np.delete(W_0317, 25, axis=0) # 删除西藏
WoP_0317 = W_0317 / P_0317[:-1, :] # 除以价格指数得到税前实际收入
WoP_0317_world = (np.array(pd.read_excel('./data/EmpGDPChnWorld.xlsx').iloc[18:33, 2]) / np.array(pd.read_excel('./data/EmpGDPChnWorld.xlsx').iloc[1:16, 2])).reshape([15, 1]) * (WoP_0317.T @ N_02[:-1, :]) / N_02[-1].T # 用和前面一样的思路直接补充外国的税前实际收入
WoP_0317 = np.insert(WoP_0317, 30, WoP_0317_world.T, axis=0)
W_0317 = np.insert(W_0317, 30, WoP_0317[-1]*P_0317.mean(axis=0), axis=0) # 这里其实应该直接乘国外部门的价格指数，但应该影响不大

## 贸易份额
data = scio.loadmat('./data/TradeShare.mat')
Pi_02 = data['Pi02']
Pi_07 = data['Pi07']
Pi_12 = data['Pi12']

## 构造2003-2006、2008-2011年的贸易份额(同样用插值)
Pi_0312 = np.ndarray([31, 31, 10])
for t in range(0, 5, 1):
    Pi_0312[:, :, t] = Pi_02 + (Pi_07 - Pi_02) / 10 * (t + 1) # 原文程序中除以10，应该是5，有误。
for t in range(0, 5, 1):
    Pi_0312[:, :, t+5] = Pi_07 + (Pi_12 - Pi_07) / 10 * (t + 1)

In [4]:
# 导入数据(3)：参数
kappa = 1.5 # 移民弹性(偏好异质性中的形状参数)
theta = 4 # 贸易弹性(效率异质性中的形状参数)
beta = 0.87 # 消费中的非住房商品份额
alpha = 0.1 / (1 - 0.48 - 0.1) # 
phip = 0.1 # 公共支出对生产的外部性
phih = 0.1 # 公共支出对居民效用的外部性
chi = 0.5 # 公共品拥挤参数
sigma = 5 # 最终品构成中各中间品的替代弹性
Xi = (1 - beta) * (1 - Tau) + alpha / (1 - alpha) + Tau / 2 + T / 2 / (1 - alpha) / (1 - T) # 简化计算，Xi乘上wN(总工资)就得到政府总购买减去转移支付(pG-TS)

In [5]:
# 调整数据
Lp_0217 = np.vstack((Lp_02.T, Lp_0317.T)).T
Lh_0217 = np.vstack((Lh_02.T, Lh_0317.T)).T
S_0217 = np.vstack((S_02.T, S_0317.T)).T
P_0217 = np.vstack((P_02.T, P_0317.T)).T
W_0217 = np.vstack((W_02.T, W_0317.T)).T
WoP_0217 = np.vstack((WoP_02.T, WoP_0317.T)).T
Mkj_0212 = np.ndarray([31, 31, 11])
Mkj_0212[:, :, 0] = Mkj_02
Mkj_0212[:, :, 1:] = Mkj_0312
Pi_0212 = np.ndarray([31, 31, 11])
Pi_0212[:, :, 0] = Pi_02
Pi_0212[:, :, 1:] = Pi_0312

In [6]:
# 求解2002至2012年每一年的均衡
N_0212 = np.ndarray([31, 11])
G_0212 = np.ndarray([31, 11])
I_0212 = np.ones([31, 11])
X0_0212 = np.zeros([31, 11])
X_0212 = np.ones([31, 11])
R_0212 = np.ndarray([31, 11])
V_0212 = np.ndarray([31, 11])
Vkmat_0212 = np.ndarray([31, 31, 11])
Vjmat_0212 = np.ndarray([31, 31, 11])
Mukj_0212 = np.ndarray([31, 31, 11])
Gamkj_0212 = np.ndarray([31, 31, 11])

for t in range(0, 11, 1):
    
    ## (a)求解出当年各省的人口
    N_0212[:, t] = (N0_02.T @ Mkj_0212[:, :, t]).flatten()

    ## (b)商品市场出清条件，利用迭代法求解，得到各个省份的总产出

    while ((X0_0212[:, t] - X_0212[:, t]) ** 2).sum() > 1e-20: ## 检验总产出与总需求是否相等
        X0_0212[:, t] = X_0212[:, t]
        R_0212[:, t] = (1 - T) * Pi_0212[:, :, t] @ X0_0212[:, t] ## 企业的税后收入=来自各省的购买
        I_0212[:, t] = (1 - Tau) * (1 - alpha) * R_0212[:, t] ## 居民的税后收入来源与企业收入的分配
        I_0212[:, t] = I_0212[:, t] / I_0212[:, t].sum() ## 将总收入标准化为1
        X_0212[:, t] = (beta * (1 - Tau) + Xi.flatten() + S_0217[:, t] / (1 - alpha)) * I_0212[:, t] / (1 - Tau) ## 总需求(居民部门的购买+政府购买)(pcN+pG)

    ## (c)政府购买
    G_0212[:, t] = (Xi.flatten() + S_0217[:, t] / (1 - alpha)) * WoP_0217[:, t] * N_0212[:, t] ## 这里用的是现实的实际工资数据，不像前面标准化为1

    ## (d)个体的间接效用
    V_0212[:, t] = (G_0212[:, t] ** phih) * ((1 - Tau) ** beta) * (WoP_0217[:, t] ** beta) * (N_0212[:, t] ** (beta - 1 - phih * chi)) * (Lh_0217[:, t] ** (1 - beta)) / ((1 - beta) ** (1 - beta))

    ## (e)计算地区间的移民成本(这里的计算方法和我自己的思路是一样的)
    Vkmat_0212[:, :, t] = np.tile(V_0212[:, t:t+1], (1, 31))
    Vjmat_0212[:, :, t] = np.tile(V_0212[:, t:t+1].T, (31, 1))
    Mukj_0212[:, :, t] = (Mkj_0212[:, :, t] / np.tile(np.diag(Mkj_0212[:, :, t]).reshape([31, 1]), (1, 31))) ** (1 / kappa) * Vkmat_0212[:, :, t] / Vjmat_0212[:, :, t]

    ## (f)计算地区间的贸易成本(和计算移民成本的思路是类似的)
    Gamkj_0212[:, :, t] = ((np.diag(Pi_0212[:, :, t]).reshape([31, 1]) @ np.diag(Pi_0212[:, :, t]).reshape([1, 31])) / (Pi_0212[:, :, t] * Pi_0212[:, :, t].T)) ** (1 / (2 * theta))


In [7]:
# 求解每一年相对于基准情形(2002)的相对变化(hat)
dW_mat = np.ndarray([31, 10])
dP_mat = np.ndarray([31, 10])
dS_mat = np.ndarray([31, 10])
dLp_mat = np.ndarray([31, 10])
dLh_mat = np.ndarray([31, 10])
dN_mat = np.ndarray([31, 10])
dGam_mat = np.ndarray([31, 31, 10])
dMu_mat = np.ndarray([31, 31, 10])
dG_mat = np.ndarray([31, 10])
dx_mat = np.ndarray([31, 10])
interm = np.ndarray([31, 1])
dT_mat = np.ndarray([31, 10])

for t in range(1, 11, 1):
    dW_mat[:, t-1] = W_0217[:, t] / W_02.flatten()
    dP_mat[:, t-1] = P_0217[:, t] / P_02.flatten()
    dS_mat[:, t-1] = S_0217[:, t] / S_02.flatten()
    dLp_mat[:, t-1] = Lp_0217[:, t] / Lp_02.flatten()
    dLh_mat[:, t-1] = Lh_0217[:, t] / Lh_02.flatten()
    dN_mat[:, t-1] = N_0212[:, t] / N_02.flatten()
    dGam_mat[:, :, t-1] = Gamkj_0212[:, :, t] / Gamkj_0212[:, :, 0]
    dMu_mat[:, :, t-1] = Mukj_0212[:, :, t] / Mukj_0212[:, :, 0]
    dMu_mat[-1, :, t-1] = 1 ## 设定国外部门的移民成本不变
    dMu_mat[:, -1, t-1] = 1 ## 同上

    dG_mat[:, t-1] = ((1 - alpha) * Xi.flatten() / S_02.flatten() + dS_mat[:, t-1]) / ((1 - alpha) * Xi.flatten() / S_02.flatten() + 1) * dN_mat[:, t-1] * dW_mat[:, t-1] / dP_mat[:, t-1]
    
    dx_mat[:, t-1] = (dN_mat[:, t-1] / dLp_mat[:, t-1]) ** alpha * dW_mat[:, t-1] ## 投入束成本的变化，使用正文式(12)，并用式(20)和(21)替换q

    interm = (np.linalg.inv((Pi_02 * dGam_mat[:, :, t-1] ** (- theta)).T) @ dP_mat[:, t-1] ** (- theta)) ** (- 1 / theta)
    dT_mat[:, t-1] = dx_mat[:, t-1] / dG_mat[:, t-1] ** phip / interm ## 还没搞明白这是在算什么

<ipython-input-7-ffe956de1938>:23: RuntimeWarning: invalid value encountered in true_divide
  dMu_mat[:, :, t-1] = Mukj_0212[:, :, t] / Mukj_0212[:, :, 0]


In [8]:
# 将求解得到的数据保存起来
scio.savemat('part0_param.mat', {
    'M' : matlab.double(M),  ## 把数据都转化成matlab.double的数据类型，方便储存
    'TL1' : matlab.double(TL1), 
    'TL2' : matlab.double(TL2), 
    'kappa' : matlab.double(kappa), 
    'theta' : matlab.double(theta), 
    'beta' : matlab.double(beta), 
    'alpha' : matlab.double(alpha), 
    'phih' : matlab.double(phih),
    'phip' : matlab.double(phip), 
    'chi' : matlab.double(chi), 
    'sigma' : matlab.double(sigma), 
    'Xi' : matlab.double(Xi), 
    'Tau' : matlab.double(Tau), 
    'T' : matlab.double(T), 
    'LandShare' : matlab.double(LandShare)
}, do_compression=True)

scio.savemat('part0_initvalue.mat', {
    'G_0212' : matlab.double(G_0212), 
    'Gamkj_0212' : matlab.double(Gamkj_0212), 
    'I_0212' : matlab.double(I_0212),
    'Lh_0217' : matlab.double(Lh_0217),
    'Lp_0217' : matlab.double(Lp_0217),
    'Mkj_0212' : matlab.double(Mkj_0212),
    'Mukj_0212' : matlab.double(Mukj_0212),
    'N0_02' : matlab.double(N0_02),
    'N_0212' : matlab.double(N_0212),
    'P_0217' : matlab.double(P_0217),
    'Pi_0212' : matlab.double(Pi_0212),
    'R_0212' : matlab.double(R_0212),
    'S_0217' : matlab.double(S_0217),
    'V_0212' : matlab.double(V_0212),
    'Vjmat_0212' : matlab.double(Vjmat_0212),
    'Vkmat_0212' : matlab.double(Vkmat_0212),
    'W_0217' : matlab.double(W_0217),
    'WoP_0217' : matlab.double(WoP_0217),
    'X0_0212' : matlab.double(X0_0212),
    'X_0212' : matlab.double(X_0212),
    'dG_mat' : matlab.double(dG_mat),
    'dGam_mat' : matlab.double(dGam_mat),
    'dLh_mat' : matlab.double(dLh_mat),
    'dLp_mat' : matlab.double(dLp_mat),
    'dMu_mat' : matlab.double(dMu_mat),
    'dN_mat' : matlab.double(dN_mat),
    'dP_mat' : matlab.double(dP_mat),
    'dS_mat' : matlab.double(dS_mat),
    'dT_mat' : matlab.double(dT_mat),
    'dW_mat' : matlab.double(dW_mat),
    'dx_mat' : matlab.double(dx_mat),
}, do_compression=True)